# Spark Server

In [ ]:
# Instalación de la API de traducción
!pip install google_trans_new

In [ ]:
import findspark
findspark.init()
import pyspark
import traceback
import json
from pyspark.sql.session import SparkSession
import os
from google_trans_new import google_translator  
import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.ml import PipelineModel
from pyspark.ml.classification import LogisticRegressionModel

conf = pyspark.SparkConf()
conf.set("spark.executor.extraClassPath",  os.path.join(os.getcwd(), 'extras/sqlite-jdbc-3.34.0.jar'))
conf.set("spark.driver.extraClassPath", os.path.join(os.getcwd(), 'extras/sqlite-jdbc-3.34.0.jar'))

sc = pyspark.SparkContext(appName="TwitterStreamApp", conf=conf)

spark = SparkSession(sc)
translator = google_translator()  

In [ ]:
# Conexión con la base de datos SQLito
url = 'jdbc:sqlite:' + os.path.join(os.getcwd(), 'database.sqlite')

df = spark.read.jdbc(url, 'Tweet')
df.createOrReplaceTempView('Tweet')

In [ ]:
# Establecemos el path de Java (no es obligatorio)
!export JAVA_HOME='/usr/lib/jvm/java-8-openjdk-amd64'
!export PATH=$JAVA_HOME/bin:$PATH

In [ ]:
# Función para analizar el sentimiento con vaderSentiment

def sentiment_score(tweet):
  
    sid = SentimentIntensityAnalyzer()
  
    sentiment = sid.polarity_scores(tweet)
      
    if sentiment['compound'] >= 0.05:
        return "Positive"
    elif sentiment['compound'] <= - 0.05:
        return "Negative"
    else:
        return 'Neutral'

In [5]:
# Función para procesar los RDD con los tweets
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time, rdd):
    print("--------- %s ---------" % str(time))
    try:
        sql_context = get_sql_context_instance(rdd.context)
        # Se importan los modelos de ML
        rf = PipelineModel.load("models/final_idf.model")
        lr = LogisticRegressionModel.load("models/final_lr.model")
        
        # Creamos un objeto Row por cada elemento del RDD con los datos que vamos a guardar en la BBDD
        row_rdd = rdd.map(lambda x: (x, translator.translate(
                x['extended_tweet']['full_text'] if x['truncated'] else x['text'], lang_src='es', lang_tgt='en'
            ))).map(lambda x: Row(
            name=x[0]['user']['screen_name'],
            date=datetime.datetime.fromtimestamp(int(x[0]['timestamp_ms']) / 1000).strftime('%Y-%m-%d %H:%M:%S'),
            text=x[0]['extended_tweet']['full_text'] if x[0]['truncated'] else x[0]['text'],
            transtext=x[1],
            sentiment=sentiment_score(x[1])
        ))
        
        # Creamos el dataframe y lo pasamos por el modelo
        tweets_df = sql_context.createDataFrame(row_rdd)
        prediction = lr.transform(rf.transform(tweets_df)).drop('words').drop('tf')\
            .drop('features').drop('rawPrediction').drop('probability').drop('label').drop('tokens')
        prediction.show()
        
        # Se guarda el dataframe resutante en la BBDD
        prediction.write.mode("append").jdbc(url, 'Tweet')
    except Exception as ex:
        pass
        #traceback.print_exception(type(ex), ex, ex.__traceback__)

In [6]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests

In [ ]:
# Configuración del servidor

sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 2)
ssc.checkpoint("checkpoint_TwitterApp")
dataStream = ssc.socketTextStream("localhost",9008)
tweets = dataStream.map(lambda x: json.loads(x))

tweets.foreachRDD(process_rdd)
mySsc = ssc

ssc.start()
ssc.awaitTermination()


----------- 2021-05-20 18:01:26 -----------
----------- 2021-05-20 18:01:28 -----------
----------- 2021-05-20 18:01:30 -----------
----------- 2021-05-20 18:01:32 -----------
----------- 2021-05-20 18:01:34 -----------
+--------------+-------------------+--------------------+--------------------+---------+----------+
|          name|               date|                text|           transtext|sentiment|prediction|
+--------------+-------------------+--------------------+--------------------+---------+----------+
|HotelEuropamad|2021-05-20 18:01:28|Museo de America
...|Museum of America...| Positive|       1.0|
|       RiggiRo|2021-05-20 18:01:28|@deborahciencia @...|@deborahciencia @...| Negative|       1.0|
+--------------+-------------------+--------------------+--------------------+---------+----------+

----------- 2021-05-20 18:01:36 -----------
+--------------+-------------------+--------------------+--------------------+---------+----------+
|          name|               date

----------- 2021-05-20 18:02:04 -----------
+--------------+-------------------+--------------------+--------------------+---------+----------+
|          name|               date|                text|           transtext|sentiment|prediction|
+--------------+-------------------+--------------------+--------------------+---------+----------+
|     viradrada|2021-05-20 18:01:56|Canción para cant...|Song to sing in t...|  Neutral|       1.0|
|       gprosam|2021-05-20 18:01:56|@AliciaM55217117 ...|@ AliciaM55217117...| Positive|       1.0|
|blanca_berjano|2021-05-20 18:01:57|Qué ganitas de es...|What Ganites from...| Positive|       1.0|
+--------------+-------------------+--------------------+--------------------+---------+----------+

----------- 2021-05-20 18:02:06 -----------
+---------+-------------------+--------------------+--------------------+---------+----------+
|     name|               date|                text|           transtext|sentiment|prediction|
+---------+----------

+-----------+-------------------+--------------------+--------------------+---------+----------+
|       name|               date|                text|           transtext|sentiment|prediction|
+-----------+-------------------+--------------------+--------------------+---------+----------+
|hiphop_xavi|2021-05-20 18:02:31|Si no lloraste co...|If you did not cr...|  Neutral|       0.0|
+-----------+-------------------+--------------------+--------------------+---------+----------+

----------- 2021-05-20 18:02:40 -----------
+-----------+-------------------+--------------------+--------------------+---------+----------+
|       name|               date|                text|           transtext|sentiment|prediction|
+-----------+-------------------+--------------------+--------------------+---------+----------+
|VidayComida|2021-05-20 18:02:32|@gastroguiasal De...|@ GastroGuiasal d...| Positive|       1.0|
|  inigosuna|2021-05-20 18:02:32|Los consejeros de...|Susanchidaz heade...|  Neutr

----------- 2021-05-20 18:08:00 -----------
----------- 2021-05-20 18:08:02 -----------
----------- 2021-05-20 18:08:04 -----------
----------- 2021-05-20 18:08:06 -----------
----------- 2021-05-20 18:08:08 -----------
----------- 2021-05-20 18:08:10 -----------
----------- 2021-05-20 18:08:12 -----------
----------- 2021-05-20 18:08:14 -----------
----------- 2021-05-20 18:08:16 -----------
----------- 2021-05-20 18:08:18 -----------
----------- 2021-05-20 18:08:20 -----------
----------- 2021-05-20 18:08:22 -----------
----------- 2021-05-20 18:08:24 -----------
----------- 2021-05-20 18:08:26 -----------
----------- 2021-05-20 18:08:28 -----------
----------- 2021-05-20 18:08:30 -----------
----------- 2021-05-20 18:08:32 -----------
----------- 2021-05-20 18:08:34 -----------
----------- 2021-05-20 18:08:36 -----------
----------- 2021-05-20 18:08:38 -----------
----------- 2021-05-20 18:08:40 -----------
----------- 2021-05-20 18:08:42 -----------
----------- 2021-05-20 18:08:44 

----------- 2021-05-20 18:14:14 -----------
----------- 2021-05-20 18:14:16 -----------
----------- 2021-05-20 18:14:18 -----------
----------- 2021-05-20 18:14:20 -----------
----------- 2021-05-20 18:14:22 -----------
----------- 2021-05-20 18:14:24 -----------
----------- 2021-05-20 18:14:26 -----------
----------- 2021-05-20 18:14:28 -----------
----------- 2021-05-20 18:14:30 -----------
----------- 2021-05-20 18:14:32 -----------
----------- 2021-05-20 18:14:34 -----------
----------- 2021-05-20 18:14:36 -----------
----------- 2021-05-20 18:14:38 -----------
----------- 2021-05-20 18:14:40 -----------
----------- 2021-05-20 18:14:42 -----------
----------- 2021-05-20 18:14:44 -----------
----------- 2021-05-20 18:14:46 -----------
----------- 2021-05-20 18:14:48 -----------
----------- 2021-05-20 18:14:50 -----------
----------- 2021-05-20 18:14:52 -----------
----------- 2021-05-20 18:14:54 -----------
----------- 2021-05-20 18:14:56 -----------
----------- 2021-05-20 18:14:58 

----------- 2021-05-20 18:20:28 -----------
----------- 2021-05-20 18:20:30 -----------
----------- 2021-05-20 18:20:32 -----------
----------- 2021-05-20 18:20:34 -----------
----------- 2021-05-20 18:20:36 -----------
----------- 2021-05-20 18:20:38 -----------
----------- 2021-05-20 18:20:40 -----------
----------- 2021-05-20 18:20:42 -----------
----------- 2021-05-20 18:20:44 -----------
----------- 2021-05-20 18:20:46 -----------
----------- 2021-05-20 18:20:48 -----------
----------- 2021-05-20 18:20:50 -----------
----------- 2021-05-20 18:20:52 -----------
----------- 2021-05-20 18:20:54 -----------
----------- 2021-05-20 18:20:56 -----------
----------- 2021-05-20 18:20:58 -----------
----------- 2021-05-20 18:21:00 -----------
----------- 2021-05-20 18:21:02 -----------
----------- 2021-05-20 18:21:04 -----------
----------- 2021-05-20 18:21:06 -----------
----------- 2021-05-20 18:21:08 -----------
----------- 2021-05-20 18:21:10 -----------
----------- 2021-05-20 18:21:12 

----------- 2021-05-20 18:26:42 -----------
----------- 2021-05-20 18:26:44 -----------
----------- 2021-05-20 18:26:46 -----------
----------- 2021-05-20 18:26:48 -----------
----------- 2021-05-20 18:26:50 -----------
----------- 2021-05-20 18:26:52 -----------
----------- 2021-05-20 18:26:54 -----------
----------- 2021-05-20 18:26:56 -----------
----------- 2021-05-20 18:26:58 -----------
----------- 2021-05-20 18:27:00 -----------
----------- 2021-05-20 18:27:02 -----------
----------- 2021-05-20 18:27:04 -----------
----------- 2021-05-20 18:27:06 -----------
----------- 2021-05-20 18:27:08 -----------
----------- 2021-05-20 18:27:10 -----------
----------- 2021-05-20 18:27:12 -----------
----------- 2021-05-20 18:27:14 -----------
----------- 2021-05-20 18:27:16 -----------
----------- 2021-05-20 18:27:18 -----------
----------- 2021-05-20 18:27:20 -----------
----------- 2021-05-20 18:27:22 -----------
----------- 2021-05-20 18:27:24 -----------
----------- 2021-05-20 18:27:26 

----------- 2021-05-20 18:32:56 -----------
----------- 2021-05-20 18:32:58 -----------
----------- 2021-05-20 18:33:00 -----------
----------- 2021-05-20 18:33:02 -----------
----------- 2021-05-20 18:33:04 -----------
----------- 2021-05-20 18:33:06 -----------
----------- 2021-05-20 18:33:08 -----------
----------- 2021-05-20 18:33:10 -----------
----------- 2021-05-20 18:33:12 -----------
----------- 2021-05-20 18:33:14 -----------
----------- 2021-05-20 18:33:16 -----------
----------- 2021-05-20 18:33:18 -----------
----------- 2021-05-20 18:33:20 -----------
----------- 2021-05-20 18:33:22 -----------
----------- 2021-05-20 18:33:24 -----------
----------- 2021-05-20 18:33:26 -----------
----------- 2021-05-20 18:33:28 -----------
----------- 2021-05-20 18:33:30 -----------
----------- 2021-05-20 18:33:32 -----------
----------- 2021-05-20 18:33:34 -----------
----------- 2021-05-20 18:33:36 -----------
----------- 2021-05-20 18:33:38 -----------
----------- 2021-05-20 18:33:40 

----------- 2021-05-20 18:39:10 -----------
----------- 2021-05-20 18:39:12 -----------
----------- 2021-05-20 18:39:14 -----------
----------- 2021-05-20 18:39:16 -----------
----------- 2021-05-20 18:39:18 -----------
----------- 2021-05-20 18:39:20 -----------
----------- 2021-05-20 18:39:22 -----------
----------- 2021-05-20 18:39:24 -----------
----------- 2021-05-20 18:39:26 -----------
----------- 2021-05-20 18:39:28 -----------
----------- 2021-05-20 18:39:30 -----------
----------- 2021-05-20 18:39:32 -----------
----------- 2021-05-20 18:39:34 -----------
----------- 2021-05-20 18:39:36 -----------
----------- 2021-05-20 18:39:38 -----------
----------- 2021-05-20 18:39:40 -----------
----------- 2021-05-20 18:39:42 -----------
----------- 2021-05-20 18:39:44 -----------
----------- 2021-05-20 18:39:46 -----------
----------- 2021-05-20 18:39:48 -----------
----------- 2021-05-20 18:39:50 -----------
----------- 2021-05-20 18:39:52 -----------
----------- 2021-05-20 18:39:54 

----------- 2021-05-20 18:45:24 -----------
----------- 2021-05-20 18:45:26 -----------
----------- 2021-05-20 18:45:28 -----------
----------- 2021-05-20 18:45:30 -----------
----------- 2021-05-20 18:45:32 -----------
----------- 2021-05-20 18:45:34 -----------
----------- 2021-05-20 18:45:36 -----------
----------- 2021-05-20 18:45:38 -----------
----------- 2021-05-20 18:45:40 -----------
----------- 2021-05-20 18:45:42 -----------
----------- 2021-05-20 18:45:44 -----------
----------- 2021-05-20 18:45:46 -----------
----------- 2021-05-20 18:45:48 -----------
----------- 2021-05-20 18:45:50 -----------
----------- 2021-05-20 18:45:52 -----------
----------- 2021-05-20 18:45:54 -----------
----------- 2021-05-20 18:45:56 -----------
----------- 2021-05-20 18:45:58 -----------
----------- 2021-05-20 18:46:00 -----------
----------- 2021-05-20 18:46:02 -----------
----------- 2021-05-20 18:46:04 -----------
----------- 2021-05-20 18:46:06 -----------
----------- 2021-05-20 18:46:08 

In [8]:
ssc.stop(False)